In [319]:
import sys
import os
import json
import pickle
import shutil
import numpy as np
import pandas as pd


class SingleShot(object):
    """docstring for singleShot"""
    def __init__(self, id,
                 origRGB_dir, json_dir, rawBMPs_dirs, CSVfull_dir, 
                 CSVcrop_dir, CSVspec_dir, RGBwithPOV_dir, BMPwithPOV_dir):
        self.id       = id
        
        self.origRGB_dir      = origRGB_dir
        self.json_dir         = json_dir
        self.rawBMPs_dirs     = rawBMPs_dirs
        self.numBMPs          = len(self.rawBMPs_dirs) if self.rawBMPs_dirs else 0
        
        self.CSVcrop_dir    = CSVcrop_dir
        self.CSVfull_dir    = CSVfull_dir
        self.CSVspec_dir    = CSVspec_dir
        self.RGBwithPOV_dir = RGBwithPOV_dir
        self.BMPwithPOV_dir = BMPwithPOV_dir
        
        self.jsonDic = None
        if self.json_dir != None:
            if os.path.isfile(self.json_dir):
                file = open(self.json_dir, 'r', encoding = 'utf-8')
                self.jsonDic = json.loads(file.read())
                file.close()



In [320]:
class SetFormatter():
    def __init__(self, directory, typeDictionary = None, from_gry = False):
        self.dir = directory
        self.dir_pcGen = os.path.join(self.dir, 'pcGen')
        self.dir_pickle = os.path.join(self.dir, 'pcGen', 'dir_pickle.gry')
        self.dir_pcGen_ref = os.path.join(self.dir, 'pcGen', 'reflection.gry')
        if not from_gry:
            self.Types = {}          #mostly associated with the downloading types
            self.usingTypes = {}     #mostly associated with the types you want to use in your work
            self.allFnames = {}
            self.setWhichTypes(typeDictionary)
            self.checkFiles()
            self.usingUniqueID_inSeperateBuckets = []
            self.numShots = 0
            self.shots = []
            self.ratio = pd.read_csv("ratio.csv")

            self.from_webGen  = 0
            self.from_rawBMPs = 1
        else:
            self.load(self.dir_pickle)
        
    def setWhichTypes(self, tmpdict):
        self.Types['origRGB']    = tmpdict['origRGB']    if ("origRGB"    in tmpdict) else False
        self.Types['json']       = tmpdict['json']       if ("json"       in tmpdict) else False
        self.Types['rawBMPs']    = tmpdict['rawBMPs']    if ("rawBMPs"    in tmpdict) else False
        self.Types['CSVfull']    = tmpdict['CSVfull']    if ("CSVfull"    in tmpdict) else False
        self.Types['CSVcrop']    = tmpdict['CSVcrop']    if ("CSVcrop"    in tmpdict) else False
        self.Types['CSVspec']    = tmpdict['CSVspec']    if ("CSVspec"    in tmpdict) else False
        self.Types['RGBwithPOV'] = tmpdict['RGBwithPOV'] if ("RGBwithPOV" in tmpdict) else False
        self.Types['BMPwithPOV'] = tmpdict['BMPwithPOV'] if ("BMPwithPOV" in tmpdict) else False
        
    def dir_origRGB(self, ID):
        os.path.join(directory, '{}.jpg'.format(ID))
    def dir_json(self, ID):
        os.path.join(directory, '{}.json'.format(ID))
    def dir_rawBMPs(self, ID, i_th):
        os.path.join(directory, 'rawBMPs', '{}_{}.bmp'.format(ID,i_th))
    def dir_CSVfull(self, ID):
        os.path.join(directory, 'webGen', '{}_full.csv'.format(ID))
    def dir_CSVcrop(self, ID):
        os.path.join(directory, 'webGen', '{}_crop.csv'.format(ID))
    def dir_CSVspec(self, ID):
        os.path.join(directory, 'webGen', '{}_spec_NDVI.csv'.format(ID))
    def dir_RGBwithPOV(self, ID):
        os.path.join(directory, 'webGen', '{}.jpeg'.format(ID))
    def dir_BMPwithPOV(self, ID):
        os.path.join(directory, 'webGen', '{}.png'.format(ID))
    
    def checkFiles(self):
        self.allFnames['origRGB']    = [s for s in os.listdir(self.dir) if os.path.isfile(os.path.join(self.dir, s)) and '.jpg' in s]
        self.allFnames['json']       = [s for s in os.listdir(self.dir) if os.path.isfile(os.path.join(self.dir, s)) and '.json' in s]
        self.allFnames['rawBMPs']    = [s for s in os.listdir(os.path.join(self.dir,'rawBMPs')) if os.path.isfile(os.path.join(self.dir,'rawBMPs',s)) and '.bmp'           in s]
        self.allFnames['CSVfull']    = [s for s in os.listdir(os.path.join(self.dir, 'webGen')) if os.path.isfile(os.path.join(self.dir, 'webGen',s)) and '_full.csv'      in s]
        self.allFnames['CSVcrop']    = [s for s in os.listdir(os.path.join(self.dir, 'webGen')) if os.path.isfile(os.path.join(self.dir, 'webGen',s)) and '_crop.csv'      in s]
        self.allFnames['CSVspec']    = [s for s in os.listdir(os.path.join(self.dir, 'webGen')) if os.path.isfile(os.path.join(self.dir, 'webGen',s)) and '_spec_NDVI.csv' in s]
        self.allFnames['RGBwithPOV'] = [s for s in os.listdir(os.path.join(self.dir, 'webGen')) if os.path.isfile(os.path.join(self.dir, 'webGen',s)) and '.jpeg'          in s]
        self.allFnames['BMPwithPOV'] = [s for s in os.listdir(os.path.join(self.dir, 'webGen')) if os.path.isfile(os.path.join(self.dir, 'webGen',s)) and '.png'           in s]
    
    def cleanFiles(self):
        for s in os.listdir(self.dir):
            ss = os.path.join(self.dir, s)

            if os.path.isfile(ss):
                if not (s in self.allFnames['origRGB'] or 
                        s in self.allFnames['json']):
                    print("removed file:", s)
                    os.remove(ss)
            if os.path.isdir(ss):
                if (not s == "rawBMPs") and (not s == "webGen"):
                    shutil.rmtree(ss)
                    print("removed directory:", s)
                    
        tmpdir = os.path.join(self.dir, 'rawBMPs')
        for s in os.listdir(tmpdir):
            ss = os.path.join(tmpdir, s)
            if os.path.isfile(ss):
                if not (s in self.allFnames['rawBMPs']):
                    print("removed file: rawBMPs/", s)
                    os.remove(ss)                
            if os.path.isdir(ss):
                shutil.rmtree(ss)
                print("removed directory: rawBMPs/", s) 
        
        tmpdir = os.path.join(self.dir, 'webGen')
        for s in os.listdir(tmpdir):
            ss = os.path.join(tmpdir, s)
            if os.path.isfile(ss):
                if not (s in self.allFnames['CSVfull'] or 
                        s in self.allFnames['CSVcrop'] or 
                        s in self.allFnames['CSVspec'] or 
                        s in self.allFnames['RGBwithPOV'] or 
                        s in self.allFnames['BMPwithPOV']):
                    print("removed file: webGen/", s)
                    os.remove(ss)                
            if os.path.isdir(ss):
                shutil.rmtree(ss)
                print("removed directory: webGen/", s) 
    def read_ids(self, tmpdict, display = False):
        self.usingTypes['origRGB']    = tmpdict['origRGB']    if ("origRGB"    in tmpdict) else False
        self.usingTypes['json']       = tmpdict['json']       if ("json"       in tmpdict) else False
        self.usingTypes['rawBMPs']    = tmpdict['rawBMPs']    if ("rawBMPs"    in tmpdict) else False
        self.usingTypes['CSVfull']    = tmpdict['CSVfull']    if ("CSVfull"    in tmpdict) else False
        self.usingTypes['CSVcrop']    = tmpdict['CSVcrop']    if ("CSVcrop"    in tmpdict) else False
        self.usingTypes['CSVspec']    = tmpdict['CSVspec']    if ("CSVspec"    in tmpdict) else False
        self.usingTypes['RGBwithPOV'] = tmpdict['RGBwithPOV'] if ("RGBwithPOV" in tmpdict) else False
        self.usingTypes['BMPwithPOV'] = tmpdict['BMPwithPOV'] if ("BMPwithPOV" in tmpdict) else False
        
        self.usingUniqueID_inSeperateBuckets = []  #unique IDs over all set files (in seperate in their file types)
        if self.usingTypes['origRGB']:
            self.usingUniqueID_inSeperateBuckets.append([fname.split('.')[0]  for fname in self.allFnames['origRGB']])
            print(("origRGB   :unique count: {}\n".format(len(self.usingUniqueID_inSeperateBuckets[-1])) if display else ''),  end='')
        if self.usingTypes['json']:
            self.usingUniqueID_inSeperateBuckets.append([fname.split('.')[0]  for fname in self.allFnames['json']])
            print(("json      :unique count: {}\n".format(len(self.usingUniqueID_inSeperateBuckets[-1])) if display else ''),  end='')
        if self.usingTypes['CSVfull']:
            self.usingUniqueID_inSeperateBuckets.append([fname.split('_')[0] + '_'+ fname.split('_')[1]  for fname in self.allFnames['CSVfull']])
            print(("CSVfull   :unique count: {}\n".format(len(self.usingUniqueID_inSeperateBuckets[-1])) if display else ''),  end='')
        if self.usingTypes['CSVcrop']:
            self.usingUniqueID_inSeperateBuckets.append([fname.split('_')[0] + '_'+ fname.split('_')[1]  for fname in self.allFnames['CSVcrop']])
            print(("CSVcrop   :unique count: {}\n".format(len(self.usingUniqueID_inSeperateBuckets[-1])) if display else ''),  end='')
        if self.usingTypes['CSVspec']:
            self.usingUniqueID_inSeperateBuckets.append([fname.split('_')[0] + '_'+ fname.split('_')[1]  for fname in self.allFnames['CSVspec']])
            print(("CSVspec   :unique count: {}\n".format(len(self.usingUniqueID_inSeperateBuckets[-1])) if display else ''),  end='')
        if self.usingTypes['RGBwithPOV']:
            self.usingUniqueID_inSeperateBuckets.append([fname.split('.')[0]  for fname in self.allFnames['RGBwithPOV']])
            print(("RGBwithPOV:unique count: {}\n".format(len(self.usingUniqueID_inSeperateBuckets[-1])) if display else ''),  end='')
        if self.usingTypes['BMPwithPOV']:
            self.usingUniqueID_inSeperateBuckets.append([fname.split('.')[0]  for fname in self.allFnames['BMPwithPOV']])
            print(("BMPwithPOV:unique count: {}\n".format(len(self.usingUniqueID_inSeperateBuckets[-1])) if display else ''),  end='')
        if self.usingTypes['rawBMPs']:
            self.usingUniqueID_inSeperateBuckets.append(list(set([fname.split('_')[0] + '_'+ fname.split('_')[1]  for fname in self.allFnames['rawBMPs']])))
            print(("rawBMPs   :unique count: {}\n".format(len(self.usingUniqueID_inSeperateBuckets[-1])) if display else ''),  end='')
            #self.usingUniqueID_inSeperateBuckets.append([fname.split('.')[0]  for fname in self.allFnames['rawBMPs']])
            #since rawBMPs have multiple files in a single shot, and I want only ID's so I did following
        
        #now calculating unique IDs over all set of files
        print(("============================\n" if display else ''),  end='')

        self.usingUniqueIDs = []
        Tmp = len(self.usingUniqueID_inSeperateBuckets)
        for i in range(len(self.usingUniqueID_inSeperateBuckets[0])):
            tmpID = self.usingUniqueID_inSeperateBuckets[0][i]
            tmpbool = True
            for t in range(1,Tmp):
                tmpbool = tmpbool and (tmpID in self.usingUniqueID_inSeperateBuckets[t])
                if not tmpbool:
                    print(("- incomplete shot: {} has no {}\n".format(tmpID, list(tmpdict.keys())[t]) if display else ''),  end='')
                    break
            if tmpbool:
                self.usingUniqueIDs.append(tmpID)
           
        self.usingUniqueIDs.sort()
        self.numShots = len(self.usingUniqueIDs)
        print(("==============================================\nIn Given types, {} unique shots in this Set.\n".format(self.numShots) if display else ''),  end='')
        
        
        for i in range(self.numShots):
            tmpid       = self.usingUniqueIDs[i]
            tmp_dirorigRGB = os.path.join(self.dir,  '{}.jpg'.format(tmpid)) if self.usingTypes['origRGB']    else None
            tmp_dirjson    = os.path.join(self.dir, '{}.json'.format(tmpid)) if self.usingTypes['json']       else None
            tmp_dirrawBMPs = [(os.path.join(sf.dir,'rawBMPs', fnamebmps)) for fnamebmps in os.listdir(os.path.join(sf.dir,'rawBMPs')) if tmpid in fnamebmps] 

            #tmp_dirrawBMPs = os.path.join(self.dir,'rawBMPs', '{}.jpeg'.format(tmpid)) if self.usingTypes['rawBMPs']    else None
            tmp_dirCSVfull = os.path.join(self.dir,'webGen',  '{}_full.csv'.format(tmpid)) if self.usingTypes['CSVfull']    else None
            tmp_dirCSVcrop = os.path.join(self.dir,'webGen',  '{}_crop.csv'.format(tmpid)) if self.usingTypes['CSVcrop']    else None
            tmp_dirCSVspec = os.path.join(self.dir,'webGen',  '{}_spec_NDVI.csv'.format(tmpid)) if self.usingTypes['CSVspec']    else None
            tmp_RGBwithPOV = os.path.join(self.dir,'webGen',  '{}.jpeg'.format(tmpid)) if self.usingTypes['RGBwithPOV'] else None
            tmp_BMPwithPOV = os.path.join(self.dir,'webGen',   '{}.png'.format(tmpid)) if self.usingTypes['BMPwithPOV'] else None
            self.shots.append(SingleShot(id             = tmpid,
                                         origRGB_dir    = tmp_dirorigRGB, 
                                         json_dir       = tmp_dirjson, 
                                         rawBMPs_dirs   = tmp_dirrawBMPs, 
                                         CSVfull_dir    = tmp_dirCSVfull, 
                                         CSVcrop_dir    = tmp_dirCSVcrop,
                                         CSVspec_dir    = tmp_dirCSVspec,
                                         RGBwithPOV_dir = tmp_RGBwithPOV,
                                         BMPwithPOV_dir = tmp_BMPwithPOV,
                                        ))
    def mkdir_pcGen(self):
        if not (os.path.isdir(self.dir_pcGen)):
            os.mkdir(self.dir_pcGen)
            
    def compose_pcGen_setWave(self, source, display = False):
        self.mkdir_pcGen()
        if self.numShots == 0:
            print("None shots in the folder")
            return
        if (source == self.from_webGen):
            print(("Composing wavelengths from webGen\n" if display else ''), end = '')
            self.setWave = np.zeros((self.numShots, 1,280))
            for i in range(self.numShots):
                tmpPdWeb = pd.read_csv(self.shots[i].CSVfull_dir)
                self.setWave[i,0,:]  = np.array(pd.read_csv(self.shots[i].CSVfull_dir)['WAVELENGTH'])
            np.save(os.path.join(self.dir_pcGen, 'setWave'), self.setWave)
            print(("Saved as pcGen/setWave.npy\n" if display else ''), end = '')
            return
        elif (source == self.from_rawBMPs):
            pass
        
    def compose_pcGen_setGray(self, source, display = False):
        self.mkdir_pcGen()
        if self.numShots == 0:
            print("None shots in the folder")
            return
        if (source == self.from_webGen):
            print(("Composing Gray pixel values from webGen\n" if display else ''), end = '')
            self.setGray = np.zeros((self.numShots, 1,280))
            for i in range(self.numShots):
                tmpPdWeb = pd.read_csv(self.shots[i].CSVfull_dir)
                self.setGray[i,0,:]  = np.array(pd.read_csv(self.shots[i].CSVfull_dir)['GRAY'])
            np.save(os.path.join(self.dir_pcGen, 'setGray'), self.setGray)
            print(("Saved as pcGen/setGray.npy\n" if display else ''), end = '')
            return
        elif (source == self.from_rawBMPs):
            pass
        else:
            print("Mistake: <source> wrong: source = ....from_webGen or ....from_rawBMPs")
            
    def compose_pcGen_setLeaf(self, source, display = False):
        self.mkdir_pcGen()
        if self.numShots == 0:
            print("None shots in the folder")
            return
        if (source == self.from_webGen):
            print(("Composing Leaf pixel values from webGen\n" if display else ''), end = '')
            self.setLeaf = np.zeros((self.numShots, 1,280))
            for i in range(self.numShots):
                tmpPdWeb = pd.read_csv(self.shots[i].CSVfull_dir)
                self.setLeaf[i,0,:]  = np.array(pd.read_csv(self.shots[i].CSVfull_dir)['LEAF'])
            np.save(os.path.join(self.dir_pcGen, 'setLeaf'), self.setLeaf)
            print(("Saved as pcGen/setLeaf.npy\n" if display else ''), end = '')
            return
        elif (source == self.from_rawBMPs):
            pass
        else:
            print("Mistake: <source> wrong: source = ....from_webGen or ....from_rawBMPs")
    
    def compose_pcGen_setMask(self, leafRange = [16,255], grayRange = [16,255], display = False):
        self.setGray = np.load(os.path.join(self.dir_pcGen, 'setGray.npy'))
        self.setLeaf = np.load(os.path.join(self.dir_pcGen, 'setLeaf.npy'))
        maskLeaf = np.logical_and(self.setLeaf >= leafRange[0], self.setLeaf <= leafRange[1])
        maskGray = np.logical_and(self.setGray >= grayRange[0], self.setGray <= grayRange[1])
        self.setMask  = np.logical_and(maskLeaf, maskGray)
        
        np.save(os.path.join(self.dir_pcGen, 'setMask'), self.setMask)
        print(("Saved as pcGen/setMask.npy\n" if display else ''), end = '')
#         self.setMask = np.logical_and(np.where())
        
    def compose_pcGen_setWhite(self, source, BG = 10.5, display = False):
        self.mkdir_pcGen()
        if self.numShots == 0:
            print("None shots in the folder")
            return
        if (source == self.from_webGen):
            print(("Composing Leaf pixel values from webGen\n" if display else ''), end = '')
            self.setWhite  = np.zeros((self.numShots,1, 280))
            for i in range(self.numShots):
                tmpPdWeb = pd.read_csv(self.shots[i].CSVfull_dir)
                tmpGray  = np.array(pd.read_csv(self.shots[i].CSVfull_dir)['GRAY'])
                self.setWhite[i,0,:] = (tmpGray-BG)/np.array(self.ratio['GRAY/WHITE'])+BG
            np.save(os.path.join(self.dir_pcGen, 'setWhite'), self.setWhite)
            print(("Saved as pcGen/setWhite.npy\n" if display else ''), end = '')
            return
        elif (source == self.from_rawBMPs):
            pass
        else:
            print("Mistake: <source> wrong: source = ....from_webGen or ....from_rawBMPs")
    
        
    def calcReflection(self,BG=10.5):
        self.setWhite = np.load(os.path.join(self.dir_pcGen, 'setWhite.npy'))
        self.setLeaf  = np.load(os.path.join(self.dir_pcGen, 'setLeaf.npy'))
        self.setWave  = np.load(os.path.join(self.dir_pcGen, 'setWave.npy'))
        if not(self.setWhite.shape == self.setLeaf.shape):
            print("Err: shape of setWhite and setLeaf are not equal")
            return
        self.numSection = self.setLeaf.shape[1]
        self.Reflection = []
        for i in range(self.numShots):
            self.Reflection.append({})
        for i in range(self.numShots):
            for s in range(self.numSection):
                tmpUsablePixelnum = np.sum(sf.setMask[i,s,:])
                tmp = np.zeros((tmpUsablePixelnum, 2))
#                 for j in range(tmpUsablePixelnum):
                tmp[:,0] = self.setWave[i,s,:][self.setMask[i,s,:]]
                tmp[:,1] = (self.setLeaf[i,s,:][self.setMask[i,s,:]]-BG)/(self.setWhite[i,s,:][self.setMask[i,s,:]]-BG)
                self.Reflection[i]['section{}'.format(s)] = np.copy(tmp)
                self.Reflection[i]['section{}_wavelength'.format(s)] = np.copy(tmp[:,0])
                self.Reflection[i]['section{}_reflection'.format(s)] = np.copy(tmp[:,1])
#                 plt.plot(tmp[:,0], tmp[:,1])
#                 plt.show()
#         self.Reflection = [[]]*self.numShots
#         for i in range(self.numShots):
#             tmp = []
#             for s in range(self.numSection):
#                 tmpUsablePixelnum = np.sum(sf.setMask[i,s,:])
#                 tmp.append(self.setWave[i,s,:][self.setMask[i,s,:]])
#                 tmp.append((self.setLeaf[i,s,:][self.setMask[i,s,:]]-BG)/(self.setWhite[i,s,:][self.setMask[i,s,:]]-BG) )
#             self.Reflection[i]= tmp
    
        file = open(self.dir_pcGen_ref, 'wb')
        pickle.dump(self.Reflection, file)
        file.close()                
        
        
    def save(self):
        file = open(self.dir_pickle, 'wb')
        pickle.dump(self, file)
        file.close()                
    def load(self, _dir_):
        #         print(_dir_)
        file = open(self.dir_pickle, 'rb')
        tmp = pickle.load(file)
        self.__dict__ = tmp.__dict__
        #         print(type(tmp.__dict__))
        file.close()
        print('read from {}'.format(self.dir_pickle))

In [321]:
Types = {
    "origRGB"   : True,
    "json"      : True,
    "rawBMPs"   : True,
    "CSVfull"   : True,
    "CSVcrop"   : True,
    "CSVspec"   : True,
    "RGBwithPOV": True,
    "BMPwithPOV": True,
}
sf = SetFormatter('/home/garid/myGit/spectrumCatcher/spectrumCatcher/tests/dname1221/13h_', Types)
sf.read_ids(Types, display=True)

origRGB   :unique count: 326
json      :unique count: 326
CSVfull   :unique count: 326
CSVcrop   :unique count: 326
CSVspec   :unique count: 326
RGBwithPOV:unique count: 326
BMPwithPOV:unique count: 326
rawBMPs   :unique count: 326
- incomplete shot: 20191221_111708 has no json
In Given types, 325 unique shots in this Set.


In [322]:
# sf.cleanFiles()

In [323]:
sf.compose_pcGen_setWave(sf.from_webGen, True)
sf.compose_pcGen_setGray(sf.from_webGen, True)
sf.compose_pcGen_setLeaf(sf.from_webGen, True)
sf.compose_pcGen_setWhite(sf.from_webGen, BG=10.5,display=True)
sf.compose_pcGen_setMask(display = True)

Composing wavelengths from webGen
Saved as pcGen/setWave.npy
Composing Gray pixel values from webGen
Saved as pcGen/setGray.npy
Composing Leaf pixel values from webGen
Saved as pcGen/setLeaf.npy
Composing Leaf pixel values from webGen
Saved as pcGen/setWhite.npy
Saved as pcGen/setMask.npy


In [324]:
sf.ratio

,Unnamed: 0,PIXEL,WAVELENGTH,GRAY/WHITE
0,0,92,442.6,0.5002
1,1,93,444.8,0.4979
2,2,94,447.0,0.4958
3,3,95,449.2,0.4941
4,4,96,451.4,0.4927
...,...,...,...,...
275,275,367,1047.6,0.9287
276,276,368,1049.8,0.9287
277,277,369,1052.0,0.9287
278,278,370,1054.2,0.9287


In [325]:
sf.calcReflection()

In [326]:
sf.Reflection[32]["section0_reflection"] - sf.Reflection[21]["section0_reflection"]

array([-0.00022285, -0.00037644, -0.0003144 , -0.00122121, -0.00169106,
       -0.00225268, -0.00223218, -0.00174841, -0.00184722, -0.00208711,
       -0.00218135, -0.00229299, -0.00260837, -0.00278627, -0.00313114,
       -0.00283719, -0.00210801, -0.00248673, -0.00267328, -0.00267266,
       -0.0033131 , -0.00312205, -0.00357274, -0.00368769, -0.0043111 ,
       -0.00539363, -0.00667846, -0.00723104, -0.00862798, -0.00989181,
       -0.01099075, -0.01303379, -0.01465229, -0.01598439, -0.01788156,
       -0.01886088, -0.02038421, -0.02107957, -0.02217338, -0.02218336,
       -0.02279784, -0.02266146, -0.023052  , -0.02327579, -0.02289367,
       -0.02255447, -0.02245515, -0.02192602, -0.02137557, -0.02110501,
       -0.01990388, -0.01911358, -0.01750906, -0.01700403, -0.01499679,
       -0.01432999, -0.01348488, -0.01193657, -0.01061934, -0.01032318,
       -0.00910802, -0.00794498, -0.00729713, -0.00539244, -0.00514939,
       -0.00493696, -0.00460634, -0.0042677 , -0.00483978, -0.00

In [327]:
sf.shots[2].rawBMPs_dirs

['/home/garid/myGit/spectrumCatcher/spectrumCatcher/tests/dname1221/13h_/rawBMPs/20191221_111710_9.bmp',
 '/home/garid/myGit/spectrumCatcher/spectrumCatcher/tests/dname1221/13h_/rawBMPs/20191221_111710_6.bmp',
 '/home/garid/myGit/spectrumCatcher/spectrumCatcher/tests/dname1221/13h_/rawBMPs/20191221_111710_2.bmp',
 '/home/garid/myGit/spectrumCatcher/spectrumCatcher/tests/dname1221/13h_/rawBMPs/20191221_111710_8.bmp',
 '/home/garid/myGit/spectrumCatcher/spectrumCatcher/tests/dname1221/13h_/rawBMPs/20191221_111710_0.bmp',
 '/home/garid/myGit/spectrumCatcher/spectrumCatcher/tests/dname1221/13h_/rawBMPs/20191221_111710_7.bmp',
 '/home/garid/myGit/spectrumCatcher/spectrumCatcher/tests/dname1221/13h_/rawBMPs/20191221_111710_14.bmp',
 '/home/garid/myGit/spectrumCatcher/spectrumCatcher/tests/dname1221/13h_/rawBMPs/20191221_111710_1.bmp',
 '/home/garid/myGit/spectrumCatcher/spectrumCatcher/tests/dname1221/13h_/rawBMPs/20191221_111710_4.bmp',
 '/home/garid/myGit/spectrumCatcher/spectrumCatcher/te

In [330]:
for i in sf.shots[2].rawBMPs_dirs:
    print(os.path.isfile(i))

True
True
True
True
True
True
True
True
True
True
True
True
True


In [143]:
i = 3
s = 0
BG = 10.5
(sf.setLeaf[i,s,:][sf.setMask[i,s,:]]-BG)/(sf.setWhite[i,s,:][sf.setMask[i,s,:]]-BG)
sf.setLeaf[i,s,:] - sf.setWhite[i,s,:]

array([-3.12193207e+02, -3.12163222e+02, -3.13526764e+02, -3.19340220e+02,
       -3.21821346e+02, -3.24402453e+02, -3.23336739e+02, -3.18390990e+02,
       -3.15195373e+02, -3.16101295e+02, -3.18483642e+02, -3.21700258e+02,
       -3.23454426e+02, -3.24288014e+02, -3.24102824e+02, -3.18351004e+02,
       -3.07430767e+02, -3.06676260e+02, -3.11623266e+02, -3.11731441e+02,
       -3.13098541e+02, -3.10848042e+02, -3.06393378e+02, -3.03331398e+02,
       -3.01895074e+02, -3.02115347e+02, -3.02969716e+02, -3.02721678e+02,
       -2.98376591e+02, -2.92841782e+02, -2.83531104e+02, -2.81451123e+02,
       -2.84509811e+02, -2.86359558e+02, -2.85578527e+02, -2.86011649e+02,
       -2.89465386e+02, -2.90558014e+02, -2.89684000e+02, -2.89155656e+02,
       -2.84376566e+02, -2.82157192e+02, -2.80820612e+02, -2.80680891e+02,
       -2.81738956e+02, -2.82203502e+02, -2.82251075e+02, -2.83298236e+02,
       -2.81526954e+02, -2.81620594e+02, -2.85009068e+02, -2.84851033e+02,
       -2.84249990e+02, -

In [39]:
sf.setWhite[sf.setMask]

(91000,)

In [54]:
s = SetFormatter('/home/garid/myGithub/spectrumCatcher/spectrumCatcher/tests/dname1221/13h_', from_gry = True)


read from /home/garid/myGithub/spectrumCatcher/spectrumCatcher/tests/dname1221/13h_/pcGen/dir_pickle.gry


In [51]:
s.__dict__.keys()


dict_keys(['dir', 'dir_pcGen', 'dir_pickle', 'Types', 'usingTypes', 'allFnames', 'usingUniqueID_inSeperateBuckets', 'numShots', 'shots', 'ratio', 'from_webGen', 'from_rawBMPs', 'usingUniqueIDs', 'setWave', 'setGray', 'setLeaf', 'setWhite', 'setMask'])

In [52]:
s.setWhite

array([[[ 89.2285086 ,  90.18132825,  91.02978351, ...,  16.42225692,
          16.42225692,  16.42225692]],

       [[ 86.56290817,  88.44068421,  88.82459325, ...,  16.42225692,
          16.42225692,  16.42225692]],

       [[341.20771691, 341.36965254, 342.85175474, ...,  16.42225692,
          16.42225692,  16.42225692]],

       ...,

       [[387.53585233, 389.43820714, 390.65328762, ...,  16.42225692,
          16.42225692,  16.42225692]],

       [[355.62861522, 357.57103167, 358.28808659, ...,  16.42225692,
          16.42225692,  16.42225692]],

       [[390.60129282, 392.98644306, 394.01485814, ...,  16.42225692,
          16.42225692,  16.42225692]]])

In [27]:
a = [{}] *4

In [29]:
a[0]['asf'] = 2

In [30]:
a

[{'asf': 2}, {'asf': 2}, {'asf': 2}, {'asf': 2}]

In [278]:
s = [{}] * 10

In [280]:
s[1]['ds'] = 32

In [281]:
s

[{'ds': 32},
 {'ds': 32},
 {'ds': 32},
 {'ds': 32},
 {'ds': 32},
 {'ds': 32},
 {'ds': 32},
 {'ds': 32},
 {'ds': 32},
 {'ds': 32}]

In [305]:
tmp_dirrawBMPs = [(os.path.join(sf.dir,'rawBMPs', fnamebmps)) for fnamebmps in os.listdir(os.path.join(sf.dir,'rawBMPs')) if tmpid in fnamebmps] 


NameError: name 'tmpid' is not defined

In [318]:
len(None) if None else 0

0